In [ ]:
#pip install pybamm

In [1]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

In [23]:
# 初始化并联参数
R_connections = [0.01, 0.01]  # 两个电池的不同连接电阻
I_total = 20                  # 总电流

# 创建并联模型组
models = [pybamm.lithium_ion.SPMe() for _ in range(2)]
simulations = []

# 分配电流并设置参数
for i, model in enumerate(models):
    # 计算本支路电流
    I_i = (I_total / R_connections[i]) / sum(1/R for R in R_connections)
    
    # 配置参数
    params = pybamm.ParameterValues("Chen2020")
    params.update({
        "Current function [A]": I_i,
 #       "Positive electrode conductivity [S.m-1]": 0.1,  # 示例参数
  #      "Negative electrode thickness [m]": 1e-4
    })
    
    # 创建模拟实例
    sim = pybamm.Simulation(model, parameter_values=params)
    simulations.append(sim)

# 并行求解
solutions = []
t_eval = np.linspace(0, 360, 10)  # 1小时仿真
for sim in simulations:
    solutions.append(sim.solve(t_eval))

# 结果可视化
pybamm.dynamic_plot(solutions, 
    output_variables=["Voltage [V]", "Current [A]", "Electrolyte concentration [mol.m-3]"])


interactive(children=(FloatSlider(value=0.0, description='t', max=360.0, step=3.6), Output()), _dom_classes=('…

In [9]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import time

# 初始化配置
# pybamm.set_logging_level("INFO")

class ParallelPack:
    def __init__(self, num_cells, conn_resistances):
        self.num_cells = num_cells
        self.conn_resistances = conn_resistances  # 连接电阻列表
        
        # 初始化各电池模型
        self.models = [pybamm.lithium_ion.SPMe(options={"thermal": "lumped"}) 
                      for _ in range(num_cells)]
        
        # 参数差异化配置（修复参数初始化问题）
        self.params_list = []
        for i in range(num_cells):
            params = pybamm.ParameterValues("Chen2020")
            params.update({
                "Positive electrode conductivity [S.m-1]": 100 + 20*i,
                "Current collector contact resistance [Ohm.m2]": 1e-4*(1 + 0.1*i),
                "Connection resistance [Ohm]": conn_resistances[i],
                "Current function [A]": 0  # 初始化电流参数
            }, check_already_exists=False)
            self.params_list.append(params)  # 正确添加参数对象
        
        # 创建仿真对象
        self.sims = [
            pybamm.Simulation(
                model, 
                parameter_values=params,
                solver=pybamm.CasadiSolver(mode="fast")
            ) for model, params in zip(self.models, self.params_list)
        ]
        
    def _calculate_internal_resistance(self, solution):
        """从解算结果中提取动态内阻"""
        V_terminal = solution["Terminal voltage [V]"].entries[-1]
        I = solution["Current [A]"].entries[-1]
        OCV = solution["Battery open-circuit voltage [V]"].entries[-1]
        return abs((OCV - V_terminal) / I) if I != 0 else np.inf
        
    def solve(self, t_eval, total_current):
        """动态求解过程"""
        dt = t_eval[1] - t_eval[0]
        solutions = [None] * self.num_cells
        
        # 初始电流分配
        currents = np.ones(self.num_cells) * total_current / self.num_cells
        
        for i, t in enumerate(t_eval[1:]):
           
            # 更新每个电池的仿真
            nsteps = [None] * self.num_cells
            
            for cell_id in range(self.num_cells):
                # 正确设置电流参数（使用setitem方法）
                self.params_list[cell_id]["Current function [A]"] = currents[cell_id]
                
                # 重新创建仿真对象以应用新参数
                self.sims[cell_id] = pybamm.Simulation(
                    self.models[cell_id],
                    parameter_values=self.params_list[cell_id],
                    solver=pybamm.CasadiSolver(mode="fast")
                )
                
                # 步进求解
                solutions[cell_id] = self.sims[cell_id].step(dt,
                                                            starting_solution=solutions[cell_id])
                # 获取求解器统计信息
                solver_stats = self.sims[cell_id].solver.stats
                nsteps[cell_id] = int(solver_stats["nsteps"])
            
            mean = np.mean(np.array(nsteps), axis=1)
            print(mean)
                
            # 计算动态总电阻（连接电阻 + 内阻）
            total_resistances = []
            for cell_id in range(self.num_cells):
                R_conn = self.conn_resistances[cell_id]
                R_int = self._calculate_internal_resistance(solutions[cell_id])
                total_resistances.append(R_conn + R_int)
            
            # 重新分配电流（基于电导率加权）
            conductances = 1 / np.array(total_resistances)
            currents = total_current * conductances / conductances.sum()
            
            # 记录系统状态
            self._log_status(i, t, currents, total_resistances)
            
        return solutions
    
    def _log_status(self, step, t, currents, resistances):
        """实时状态输出"""
        print(f"\nStep {step} @ {t:.1f}s:")
        for cell_id in range(self.num_cells):
            print(f"Cell {cell_id+1}: I={currents[cell_id]:.2f}A, R_total={resistances[cell_id]:.4f}Ω")

# 运行参数
t_total = 2  # 总仿真时间 [s]
dt = 1  # 电流更新间隔 [s]
t_eval = np.arange(0, t_total + dt, dt)
total_current = 20  # 系统总电流 [A]

repeat = 1
# batch_sizes = [1,2,4,8,16]
batch_sizes = [1]

time_cost = np.zeros((len(batch_sizes), repeat)) 

for j in range(repeat):     
    for k in range(len(batch_sizes)):

        # 创建并联电池组实例
        pack = ParallelPack(
            num_cells=batch_sizes[k],
            conn_resistances= [i*0.0001 + 0.001 for i in range(batch_sizes[k])]   # 电池连接电阻
        )
        start = time.time()
        solutions = pack.solve(t_eval, total_current)
        end = time.time()
        
        time_cost[k,j] = float((end - start)/t_total)
        print(time_cost)
            
mean = np.mean(time_cost, axis=1)
var = np.var(time_cost, axis=1, ddof=1) 

for k in range(len(batch_sizes)):
    print(mean[k],var[k])


2025-04-02 20:06:20.674 - [INFO] base_model._build_model(652): Start building Single Particle Model with electrolyte
2025-04-02 20:06:20.734 - [INFO] base_battery_model.build_model(1073): Finish building Single Particle Model with electrolyte
2025-04-02 20:06:20.739 - [INFO] parameter_values.process_model(394): Start setting parameters for Single Particle Model with electrolyte
2025-04-02 20:06:20.816 - [INFO] parameter_values.process_model(463): Finish setting parameters for Single Particle Model with electrolyte
2025-04-02 20:06:20.817 - [INFO] discretisation.process_model(145): Start discretising Single Particle Model with electrolyte
2025-04-02 20:06:20.826 - [INFO] discretisation.remove_independent_variables_from_rhs(1126): removing variable Discharge capacity [A.h] from rhs
2025-04-02 20:06:20.827 - [INFO] discretisation.remove_independent_variables_from_rhs(1126): removing variable Throughput capacity [A.h] from rhs
2025-04-02 20:06:21.208 - [INFO] discretisation.process_model(2

AttributeError: 'CasadiSolver' object has no attribute 'stats'

In [59]:

solutions[0].plot()
solutions[1].plot()

interactive(children=(FloatSlider(value=0.0, description='t', step=1.0), Output()), _dom_classes=('widget-inte…

interactive(children=(FloatSlider(value=0.0, description='t', step=1.0), Output()), _dom_classes=('widget-inte…